In [1]:
pip install sec-api

In [7]:
from sec_api import QueryApi

api_key = QueryApi(api_key='7e49f7c644224d14d89afca6884c36c784ddb213ba513247d8d2d5a829fe6a17')


def get_filing_urls(ticker, years, api_key):
    query_api = QueryApi(api_key='7e49f7c644224d14d89afca6884c36c784ddb213ba513247d8d2d5a829fe6a17')
    urls = {}

    for year in years:
        query = {
            "query": f'ticker:"{ticker}" AND formType:"10-K" AND filedAt:[{year}-01-01 TO {year}-12-31]',
            "from": "0",
            "size": "10",
            "sort": [{"filedAt": {"order": "desc"}}]
        }
        response = query_api.get_filings(query)
        filings = response.get('filings', [])
        if filings:
            urls[year] = filings[0].get('linkToTxt', "No URL found")

    return urls

# API key and rears
api_key = 'YOUR_API_KEY'
aapl_years = [1995, 1996, 1997, 1999, 2000, 2002, 2003, 2004, 2005]
msft_years = [1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005]

# fetch URLs
aapl_urls = get_filing_urls('AAPL', aapl_years, api_key)
msft_urls = get_filing_urls('MSFT', msft_years, api_key)

print("AAPL URLs:", aapl_urls)
print("MSFT URLs:", msft_urls)



AAPL URLs: {1995: 'https://www.sec.gov/Archives/edgar/data/320193/0000320193-95-000016.txt', 1996: 'https://www.sec.gov/Archives/edgar/data/320193/0000320193-96-000023.txt', 1997: 'https://www.sec.gov/Archives/edgar/data/320193/0001047469-97-006960.txt', 1999: 'https://www.sec.gov/Archives/edgar/data/320193/000091205799010244/0000912057-99-010244.txt', 2000: 'https://www.sec.gov/Archives/edgar/data/320193/000091205700053623/0000912057-00-053623.txt', 2002: 'https://www.sec.gov/Archives/edgar/data/320193/000104746902007674/0001047469-02-007674.txt', 2003: 'https://www.sec.gov/Archives/edgar/data/320193/000104746903041604/0001047469-03-041604.txt', 2004: 'https://www.sec.gov/Archives/edgar/data/320193/000104746904035975/0001047469-04-035975.txt', 2005: 'https://www.sec.gov/Archives/edgar/data/320193/000110465905058421/0001104659-05-058421.txt'}
MSFT URLs: {1995: 'https://www.sec.gov/Archives/edgar/data/789019/0000891020-95-000433.txt', 1996: 'https://www.sec.gov/Archives/edgar/data/78901

In [10]:
from sec_api import ExtractorApi
import json

def process_filings(filing_urls, items, api_key):
    extractor_api = ExtractorApi(api_key='7e49f7c644224d14d89afca6884c36c784ddb213ba513247d8d2d5a829fe6a17')
    filings_dict = {}

    for year, url in filing_urls.items():
        filings_dict[year] = {}
        for item in items:
            try:
                extracted_text = extractor_api.get_section(url, item, "text")
                if extracted_text:
                    filings_dict[year][f"item{item}"] = extracted_text
                else:
                    filings_dict[year][f"item{item}"] = "Section not found"
            except Exception as e:
                filings_dict[year][f"item{item}"] = f"Failed to extract: {str(e)}"

    return filings_dict

# API Key and filing information
api_key = 'YOUR_API_KEY_HERE'
items = ['1', '1A', '7']  # items to extract

# URLs fetched earlier
aapl_filing_urls = aapl_urls # replace with actual AAPL URLs
msft_filing_urls = msft_urls  # replace with actual MSFT URLs

# fetch and organize data
aapl_filings = process_filings(aapl_filing_urls, items, api_key)
msft_filings = process_filings(msft_filing_urls, items, api_key)

# dump data to JSON files
with open('aapl_1995_2005.json', 'w') as f:
    json.dump(aapl_filings, f)

with open('msft_1995_2005.json', 'w') as f:
    json.dump(msft_filings, f)
